In [1]:
from astropy.table import Table, join, Column
import matplotlib.pyplot as plt
import numpy as np
# My modules
from my_progs.catalog.vsh_deg1_cor import vsh_deg01_fitting
from my_progs.catalog.vsh_deg2_cor import vsh_deg02_fitting
from my_progs.catalog.pos_diff import pos_diff_calc
from my_progs.catalog.pos_err import pos_err_calc

Read these two catalogs.

In [2]:
# read source position from VLBI and Gaia catalogs.
from my_progs.vlbi.read_sou import read_cat
tv = read_cat("/Users/Neo/Astronomy/Data/VLBISolutions/"
                "vlbi2_server/GalacticAberration/GaiaDR2-timespan-solutions/"
                "gaiadr2-timespan-noGAb.cat")

# Read Gaia DR2 IERS quasar
# tg = Table.read("/Users/Neo/Astronomy/Data/catalogs/Gaia_DR2/"
#                 "gaiadr2_iers.fits")
from my_progs.catalog.read_gaia import read_gaiadr2_qso

tg = read_gaiadr2_qso()

Then I obtained a sample of common sources by the cross-match.

In [4]:
# Cross-match
t_com = join(tv, tg, keys="iers_name", table_names=["v", "g"])

# gaiadr2-vlbi positional offsets (VLBI - Gaia DR2).
[dra, ddec, dra_err, ddec_err, dra_ddec_cor,
 ang_sep, X_a, X_d, X, X2] = pos_diff_calc(
    t_com["ra_v"], t_com["ra_err"], t_com["dec_v"], t_com["dec_err"], t_com["ra_dec_corr_v"],
    t_com["ra_g"],  t_com["ra_error"], t_com["dec_g"], t_com["dec_error"], t_com["ra_dec_corr_g"])

dra_ddec_cov = dra_ddec_cor * dra_err * ddec_err

# # degree -> rad
# ra = np.array(t_com["ra_1"])
# dec = np.array(t_com["dec_1"])
# ra_rad, dec_rad = np.deg2rad(ra), np.deg2rad(dec)

t_com.add_columns([dra, ddec, dra_err, ddec_err, dra_ddec_cor,
                   ang_sep, X_a, X_d, X, X2],
                  names=["dra", "ddec", "dra_err", "ddec_err", "dra_ddec_cov",
                         "ang_sep", "nor_dra",  "nor_ddec", "nor_sep_cov", "nor_sep"])

t_com

ivs_name,iers_name,ra_v,dec_v,ra_err,dec_err,ra_dec_corr_v,pos_err_v,mean_epo,beg_epo,end_epo,num_sess,num_del,num_delrate,flag,source_id,ra_g,ra_error,dec_g,dec_error,pos_err_g,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr_g,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,dra,ddec,dra_err,ddec_err,dra_ddec_cov,ang_sep,nor_dra,nor_ddec,nor_sep_cov,nor_sep
,,deg,deg,mas,mas,,mas,MJD,MJD,MJD,,,,,,deg,mas,deg,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,mag,mag,mag,deg,deg,mas,mas,mas,deg,deg,deg,deg,deg
str8,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,str3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0000-197,0000-197,0.8278125995,-19.4562098104,0.3149,0.653,0.461,0.6722242680190545,2014.6,2014.6,2014.6,1,67,0,GLO,2365855384650325376,0.8278123681512258,0.2387757610163772,-19.456209949035525,0.1922200644996743,0.24241216451703895,-0.45544305828221626,0.3185109654739551,0.6116734571782345,0.5735415853774198,0.37085954108422414,0.39878223344506625,0.13461179,0.052950118,0.35212186,-0.022328816,-0.097155325,-0.019389542,0.2804625,-0.1926077,-0.35069835,0.24758662,19.193932,19.635006,18.525171,0.7852964835166851,0.49908788781749536,0.39519093366238817,0.6807037190997703,0.1009736350687503,0.9304726147446792,1.9871318307812302,0.7331940076330689,1.9871789731405747,2.1180808213411035
0001+459,0001+459,1.0671985141,46.25499186,0.161,0.145,0.18,0.16833348564022674,2014.6,2014.6,2014.6,1,72,0,GLO,387054966590038656,1.0671982239853723,0.7718284266351787,46.25499167308568,0.6969576827283192,0.777985787941841,-0.029368229696100832,1.0984495193905532,0.9944652987116404,1.2381622163997819,0.4579145221304079,0.9951609005577559,-0.06277916,0.42371804,-0.60506684,0.19084497,0.07386451,0.025139727,-0.6068582,-0.4637956,0.30770168,-0.2570573,20.491125,20.788141,19.799097,0.7221592563111845,0.6728915536768909,0.7884415768857801,0.7118813184190385,-0.029568802622980596,0.9870648582973781,0.9159324894605377,0.9452299649768351,1.3522894213624823,1.3162036361974871
0002-170,0002-170,1.3247240786,-16.8012995133,0.1953,0.4598,0.18,0.46142277623777267,2014.6,2014.6,2014.6,1,72,0,GLO,2414919854088816640,1.324724085562965,0.12209945018003658,-16.8012994100183,0.09340584476751623,0.12802895680534068,-0.13861516190224235,0.17585208899659477,0.004812872190701029,0.2745290537552927,0.1572019487857582,0.18733964355104002,0.2956712,0.2617457,-0.11088492,-0.16225792,-0.1564921,-0.13369387,0.14287123,0.247167,-0.3034156,0.260597,17.744463,17.8112,17.465117,-0.023996650985147777,-0.37181411869369185,0.23032665007390535,0.46919153001384556,0.019535880719364386,0.3725876784308222,-0.10418529934529037,-0.7924570136266523,0.793452109707901,0.7992763571166984
0002-478,0002-478,1.1485645299,-47.6054455475,0.0906,0.1561,0.115,0.15661827454691143,2015.47,2014.57,2016.37,10,126,0,GLO,4978016560925910400,1.1485644787260738,0.32535001967377664,-47.60544549244506,0.38175267275937685,0.4046925099699907,-0.394507921304075,0.5551341682993441,0.10766725569112054,0.5623357858130931,-0.2990603581366252,0.6935629455335646,-0.2602696,0.38209838,-0.3261622,-0.043664124,-0.36382514,0.12289763,-0.07530141,-0.16826801,-0.2654429,-0.06584475,19.847563,19.998808,19.225542,0.12421119180790428,-0.1981977845844085,0.33772917449004436,0.41243461682905314,-0.030699923087134314,0.23390336035317574,0.36778342290226335,-0.4805556480884775,0.5504634852830819,0.6051432698720541
0003+158,0003+158,1.4968240525,16.1636171408,0.2126,0.3451,-0.075,0.3456899131203155,2016.05,2016.05,2016.05,1,61,0,GLO,2772557627594347904,1.4968240859821051,0.05

There are 1274 common sources. 
Before we investigate the systematcis, we should remove some outliers.

In [6]:
# First we remove sources with less than three observations.
mask1 = (t_com["num_del"] >=3)
t_com1 = t_com[mask1]
t_com1

ivs_name,iers_name,ra_v,dec_v,ra_err,dec_err,ra_dec_corr_v,pos_err_v,mean_epo,beg_epo,end_epo,num_sess,num_del,num_delrate,flag,source_id,ra_g,ra_error,dec_g,dec_error,pos_err_g,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr_g,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,dra,ddec,dra_err,ddec_err,dra_ddec_cov,ang_sep,nor_dra,nor_ddec,nor_sep_cov,nor_sep
,,deg,deg,mas,mas,,mas,MJD,MJD,MJD,,,,,,deg,mas,deg,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,mag,mag,mag,deg,deg,mas,mas,mas,deg,deg,deg,deg,deg
str8,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,str3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0000-197,0000-197,0.8278125995,-19.4562098104,0.3149,0.653,0.461,0.6722242680190545,2014.6,2014.6,2014.6,1,67,0,GLO,2365855384650325376,0.8278123681512258,0.2387757610163772,-19.456209949035525,0.1922200644996743,0.24241216451703895,-0.45544305828221626,0.3185109654739551,0.6116734571782345,0.5735415853774198,0.37085954108422414,0.39878223344506625,0.13461179,0.052950118,0.35212186,-0.022328816,-0.097155325,-0.019389542,0.2804625,-0.1926077,-0.35069835,0.24758662,19.193932,19.635006,18.525171,0.7852964835166851,0.49908788781749536,0.39519093366238817,0.6807037190997703,0.1009736350687503,0.9304726147446792,1.9871318307812302,0.7331940076330689,1.9871789731405747,2.1180808213411035
0001+459,0001+459,1.0671985141,46.25499186,0.161,0.145,0.18,0.16833348564022674,2014.6,2014.6,2014.6,1,72,0,GLO,387054966590038656,1.0671982239853723,0.7718284266351787,46.25499167308568,0.6969576827283192,0.777985787941841,-0.029368229696100832,1.0984495193905532,0.9944652987116404,1.2381622163997819,0.4579145221304079,0.9951609005577559,-0.06277916,0.42371804,-0.60506684,0.19084497,0.07386451,0.025139727,-0.6068582,-0.4637956,0.30770168,-0.2570573,20.491125,20.788141,19.799097,0.7221592563111845,0.6728915536768909,0.7884415768857801,0.7118813184190385,-0.029568802622980596,0.9870648582973781,0.9159324894605377,0.9452299649768351,1.3522894213624823,1.3162036361974871
0002-170,0002-170,1.3247240786,-16.8012995133,0.1953,0.4598,0.18,0.46142277623777267,2014.6,2014.6,2014.6,1,72,0,GLO,2414919854088816640,1.324724085562965,0.12209945018003658,-16.8012994100183,0.09340584476751623,0.12802895680534068,-0.13861516190224235,0.17585208899659477,0.004812872190701029,0.2745290537552927,0.1572019487857582,0.18733964355104002,0.2956712,0.2617457,-0.11088492,-0.16225792,-0.1564921,-0.13369387,0.14287123,0.247167,-0.3034156,0.260597,17.744463,17.8112,17.465117,-0.023996650985147777,-0.37181411869369185,0.23032665007390535,0.46919153001384556,0.019535880719364386,0.3725876784308222,-0.10418529934529037,-0.7924570136266523,0.793452109707901,0.7992763571166984
0002-478,0002-478,1.1485645299,-47.6054455475,0.0906,0.1561,0.115,0.15661827454691143,2015.47,2014.57,2016.37,10,126,0,GLO,4978016560925910400,1.1485644787260738,0.32535001967377664,-47.60544549244506,0.38175267275937685,0.4046925099699907,-0.394507921304075,0.5551341682993441,0.10766725569112054,0.5623357858130931,-0.2990603581366252,0.6935629455335646,-0.2602696,0.38209838,-0.3261622,-0.043664124,-0.36382514,0.12289763,-0.07530141,-0.16826801,-0.2654429,-0.06584475,19.847563,19.998808,19.225542,0.12421119180790428,-0.1981977845844085,0.33772917449004436,0.41243461682905314,-0.030699923087134314,0.23390336035317574,0.36778342290226335,-0.4805556480884775,0.5504634852830819,0.6051432698720541
0003+158,0003+158,1.4968240525,16.1636171408,0.2126,0.3451,-0.075,0.3456899131203155,2016.05,2016.05,2016.05,1,61,0,GLO,2772557627594347904,1.4968240859821051,0.05

In [9]:
# Consider X follow a Rayleigh distribution
N = len(t_com1)
X0 = np.sqrt(2 * np.log(N))

print("There are %d sources.\n"
      "When X >= %.2f, the number of the outlier is smaller than 1." % (N, X0))

# Then we remove these sources with X >= 3.8 or eema >= 10 mas
mask2 = ((t_com1["nor_sep_cov"] <= 3.8) & (t_com1["ang_sep"] <= 10) & (
    t_com1["pos_err_v"] <= 10) & (t_com1["pos_err_g"] <= 10))
t_com2 = t_com1[mask2]
t_com2

There are 1263 sources.
When X >= 3.78, the number of the outlier is smaller than 1.


ivs_name,iers_name,ra_v,dec_v,ra_err,dec_err,ra_dec_corr_v,pos_err_v,mean_epo,beg_epo,end_epo,num_sess,num_del,num_delrate,flag,source_id,ra_g,ra_error,dec_g,dec_error,pos_err_g,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr_g,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,dra,ddec,dra_err,ddec_err,dra_ddec_cov,ang_sep,nor_dra,nor_ddec,nor_sep_cov,nor_sep
,,deg,deg,mas,mas,,mas,MJD,MJD,MJD,,,,,,deg,mas,deg,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,mag,mag,mag,deg,deg,mas,mas,mas,deg,deg,deg,deg,deg
str8,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,str3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0000-197,0000-197,0.8278125995,-19.4562098104,0.3149,0.653,0.461,0.6722242680190545,2014.6,2014.6,2014.6,1,67,0,GLO,2365855384650325376,0.8278123681512258,0.2387757610163772,-19.456209949035525,0.1922200644996743,0.24241216451703895,-0.45544305828221626,0.3185109654739551,0.6116734571782345,0.5735415853774198,0.37085954108422414,0.39878223344506625,0.13461179,0.052950118,0.35212186,-0.022328816,-0.097155325,-0.019389542,0.2804625,-0.1926077,-0.35069835,0.24758662,19.193932,19.635006,18.525171,0.7852964835166851,0.49908788781749536,0.39519093366238817,0.6807037190997703,0.1009736350687503,0.9304726147446792,1.9871318307812302,0.7331940076330689,1.9871789731405747,2.1180808213411035
0001+459,0001+459,1.0671985141,46.25499186,0.161,0.145,0.18,0.16833348564022674,2014.6,2014.6,2014.6,1,72,0,GLO,387054966590038656,1.0671982239853723,0.7718284266351787,46.25499167308568,0.6969576827283192,0.777985787941841,-0.029368229696100832,1.0984495193905532,0.9944652987116404,1.2381622163997819,0.4579145221304079,0.9951609005577559,-0.06277916,0.42371804,-0.60506684,0.19084497,0.07386451,0.025139727,-0.6068582,-0.4637956,0.30770168,-0.2570573,20.491125,20.788141,19.799097,0.7221592563111845,0.6728915536768909,0.7884415768857801,0.7118813184190385,-0.029568802622980596,0.9870648582973781,0.9159324894605377,0.9452299649768351,1.3522894213624823,1.3162036361974871
0002-170,0002-170,1.3247240786,-16.8012995133,0.1953,0.4598,0.18,0.46142277623777267,2014.6,2014.6,2014.6,1,72,0,GLO,2414919854088816640,1.324724085562965,0.12209945018003658,-16.8012994100183,0.09340584476751623,0.12802895680534068,-0.13861516190224235,0.17585208899659477,0.004812872190701029,0.2745290537552927,0.1572019487857582,0.18733964355104002,0.2956712,0.2617457,-0.11088492,-0.16225792,-0.1564921,-0.13369387,0.14287123,0.247167,-0.3034156,0.260597,17.744463,17.8112,17.465117,-0.023996650985147777,-0.37181411869369185,0.23032665007390535,0.46919153001384556,0.019535880719364386,0.3725876784308222,-0.10418529934529037,-0.7924570136266523,0.793452109707901,0.7992763571166984
0002-478,0002-478,1.1485645299,-47.6054455475,0.0906,0.1561,0.115,0.15661827454691143,2015.47,2014.57,2016.37,10,126,0,GLO,4978016560925910400,1.1485644787260738,0.32535001967377664,-47.60544549244506,0.38175267275937685,0.4046925099699907,-0.394507921304075,0.5551341682993441,0.10766725569112054,0.5623357858130931,-0.2990603581366252,0.6935629455335646,-0.2602696,0.38209838,-0.3261622,-0.043664124,-0.36382514,0.12289763,-0.07530141,-0.16826801,-0.2654429,-0.06584475,19.847563,19.998808,19.225542,0.12421119180790428,-0.1981977845844085,0.33772917449004436,0.41243461682905314,-0.030699923087134314,0.23390336035317574,0.36778342290226335,-0.4805556480884775,0.5504634852830819,0.6051432698720541
0003+158,0003+158,1.4968240525,16.1636171408,0.2126,0.3451,-0.075,0.3456899131203155,2016.05,2016.05,2016.05,1,61,0,GLO,2772557627594347904,1.4968240859821051,0.05

This left 1024 common source. Actually, if we adopted the X0 to be 3.78 rather than 3.8, there would be 1023 sources left.

In [10]:
import astropy.units as u

# Genearte some arrays for estimating the ststematics
dra = np.array(t_com2["dra"])
ddec = np.array(t_com2["ddec"])
dra_err = np.array(t_com2["dra_err"])
ddec_err = np.array(t_com2["ddec_err"])
dra_ddec_cov = np.array(t_com2["dra_ddec_cov"])
ra_rad = np.array(t_com2["ra_v"].to(u.radian))
dec_rad = np.array(t_com2["dec_v"].to(u.radian))

In [14]:
# l_max = 1
w1, sig1, corrcoef1, _, _, _ = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err, 
    cov=dra_ddec_cov, elim_flag="None")

# l_max = 2
w2, sig2, corrcoef2, _, _, _ = vsh_deg02_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,  
    cov=dra_ddec_cov, elim_flag="None")

# mas -> uas
w1 = w1 * 1.e3
sig1 = sig1 * 1.e3
w2 = w2 * 1.e3
sig2 = sig2 * 1.e3

# Print results
print("Estimates (%6d sources)\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "               Rotation [uas]                 "
      "                  Glide [uas]               \n"
      "               x             y             z"
      "               x             y             z\n"
      "----------------------------------------------"
      "----------------------------------------------\n"
      "l_max=1  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "l_max=2  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  "
      "  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f  %+4.0f +/- %3.0f\n"
      "----------------------------------------------"
      "----------------------------------------------\n" %
      (dra.size, w1[3], sig1[3], w1[4], sig1[4], w1[5], sig1[5],
       w1[0], sig1[0], w1[1], sig1[1], w1[2], sig1[2],
       w2[3], sig2[3], w2[4], sig2[4], w2[5], sig2[5],
       w2[0], sig2[0], w2[1], sig2[1], w2[2], sig2[2]))

quad_names = Column(["ER22", "EI22", "ER21", "EI21", "E20",
                     "MR22", "MI22", "MR21", "MI21", "M20"])
t_quad = Table([quad_names, w2[6:], sig2[6:]], names=["Quadrupolar term", "Estimate", "Error"])
t_quad["Estimate"].format = "%5.0f"
t_quad["Error"].format = "%5.0f"
print(t_quad)

# apriori statistics (weighted)
#         mean for RA:      0.021 mas
#         wrms for RA:      0.719 mas
#          std for RA:      0.719 mas
#        mean for Dec:      0.053 mas
#        wrms for Dec:      0.948 mas
#         std for Dec:      0.946 mas

# apriori reduced Chi-square for:      1.797
# posteriori statistics  of vsh01 fit (weighted)
#         mean for RA:      0.014 mas
#          rms for RA:      0.718 mas
#          std for RA:      0.717 mas
#        mean for Dec:      0.064 mas
#         rms for Dec:      0.948 mas
#         std for Dec:      0.946 mas

# posteriori reduced Chi-square for:      1.783
# goodness-of-fit is      0.000
# apriori statistics (weighted)
#         mean for RA:      0.021 mas
#         wrms for RA:      0.719 mas
#          std for RA:      0.719 mas
#        mean for Dec:      0.053 mas
#        wrms for Dec:      0.948 mas
#         std for Dec:      0.946 mas

# apriori reduced Chi-square for:      1.797
# posteriori statistics of vsh

In [ ]:
t_com[t_com["iers_name"] == "1901+319"]